In [ ]:
!git clone https://github.com/chmiti/vision-llm-project.git
%cd vision-llm-project

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import os
os.makedirs("data/images", exist_ok=True)


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import shutil

# Déplacer toutes les images vers data/images
for filename in uploaded.keys():
    shutil.move(filename, f"data/images/{filename}")


In [ ]:
!pip install -q transformers accelerate bitsandbytes


In [ ]:
from huggingface_hub import login
login(token="hf_xxxxxxxxxxxxxxxxxxxxxxx")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import csv
import os

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

def ask_mistral(prompt):
    response = generator(prompt, max_new_tokens=150, do_sample=True, temperature=0.5)[0]['generated_text']
    return response[len(prompt):].strip()

def main(captions_file="captions.csv", output_file="reasoning.csv"):
    print("✅ Début inférence...")
    with open(captions_file, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        rows = list(reader)

    with open(output_file, "w", newline='', encoding='utf-8') as out_f:
        writer = csv.writer(out_f)
        writer.writerow(["filename", "caption", "llm_response"])

        for row in rows:
            prompt = f"""### Instruction:
You are a helpful assistant analyzing traffic images.

### Caption:
{row['caption']}

### Question:
What do you infer about this image?

### Answer:"""
            answer = ask_mistral(prompt)
            print(f"\n🖼️ {row['filename']}\n🤖 {answer}")
            writer.writerow([row['filename'], row['caption'], answer])

main()
